In [63]:
import requests
import time
import threading
import os
import io

In [64]:
URL_TO_HIT = "http://localhost:8080/function/openfaas-fn/prime-numbers/prime/50000"

# Requests

In [65]:
def make_ramp(start_count:int, end_count:int, steps:int, start_padding:int = 0, end_padding:int = 0)->list[int]:
    delta = end_count - start_count
    return [start_count for _ in range(start_padding)] + [start_count + int((i/steps)*delta) for i in range(steps+1)] + [end_count for _ in range(end_padding)]

In [66]:
def do_request():
    r = requests.get(
       URL_TO_HIT
    )

def run_requests(ramp:list[int]):
    for reqs in ramp:
      for i in range(reqs):
         threading.Thread(target=do_request).start()
      time.sleep(1)

# Analysis

In [67]:
def get_pods(output_file:io.TextIOWrapper, t:int):
    txt = os.popen("kubectl get pods -n kube-system -o name").read().strip().split("\n")[1:]
    for line in txt:
        output_file.write(str(t) + " " + line.strip().replace("pod/", "") + "\n")

def monitor_pods(file_name:str, duration:int):
    with open(file_name, "w") as f:
        for x in range(duration):
            get_pods(f, x+1)
            time.sleep(1)


# Main

In [69]:
t_anal = threading.Thread(target=monitor_pods, args=("pod_names.txt"))
t_reqs = threading.Thread(target=run_requests, args=(make_ramp(10,100,100,100,200)))

print("Starting requests")
t_reqs.start()
print("Starting analysis")
t_anal.start()
print("Started")
t_reqs.join()
print("Joined request thread")
t_anal.join()
print("Joined analysis thread")
print("Done.")



Exception in thread Exception in thread Thread-728 (monitor_pods):
Thread-729 (run_requests):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/Users/pietro/Dev/neptuneplus/neptune-experiment/.venv/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: run_requests() takes 1 positional argument but 401 were given
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1043, in _bootstrap_inn

Starting requests
Starting analysis
Started
Joined request thread
Joined analysis thread
Done.
